#### Install Selenium & Import Necessary Packages

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#### Connect Selenium to Chrome and Direct to Desired URL

In [56]:
browser = webdriver.Chrome(executable_path  = '/Users/stephaniekendall/Library/Application Support/Google/chromedriver')

In [57]:
browser.get('https://www.waterqualitydata.us/portal/#countrycode=US&sampleMedia=Water&sampleMedia=water&startDateLo=01-01-2014&startDateHi=01-01-2019&mimeType=csv&dataProfile=biological')

#### Create a Function for Downloading State CSVs

In [53]:
def get_csv(x):
    #ensure all location parameters are clear
#     clear_field = browser.find_element_by_xpath('//*[@title="Remove all items"]')
#     clear_field.click()
    
# #     # input US as Country parameter
#     country = browser.find_element_by_xpath('//*[@class="select2-search__field"]')
#     country.send_keys('US', Keys.ENTER)
    
    # input x as State parameter
    element = browser.find_element_by_xpath("//input[@placeholder='All'][1]")
    element.send_keys(x, Keys.ENTER)
    
    # Download Biological metadata sample results
    results_button = browser.find_element_by_id("biosamples")
    results_button.click()
    browser.implicitly_wait(5)
    
    # submit and download results 
    download_button = browser.find_element_by_xpath('//*[@type="submit"]')
    download_button.click()
    
    # wait until query loads to press download
    browser.implicitly_wait(15)
    continue_button = browser.find_element_by_xpath('//*[@id="progress-continue-btn"]')
    continue_button.click()

In [ ]:
<input class="select2-search__field" type="search" tabindex="0" autocomplete="off" autocorrect="off" autocapitalize="none" spellcheck="false" role="searchbox" aria-autocomplete="list" placeholder="All" style="width: 539.906px;">

In [58]:
get_csv('MN')

In [30]:
clear_field = browser.find_element_by_xpath('//*[@title="Remove all items"]')
clear_field.click()

In [59]:
from zipfile import ZipFile

In [61]:
file_name = "biologicalresult.zip"

In [64]:
def unzipfile(x):
    with ZipFile(x, 'r') as zip:
        zip.printdir()
        zip.extractall()

In [65]:
unzipfile('biologicalresult1.zip')

File Name                                             Modified             Size
biologicalresult.csv                           2019-11-12 16:39:30    176633921


In [74]:
pip install os
pip install shutil

SyntaxError: invalid syntax (<ipython-input-74-4692f12dfdfe>, line 2)

In [75]:
import os
import shutil

path = 'C:\\Users\stephaniekendall\Downloads'

source = 'C:\\Users\stephaniekendall\Downloads\biologicalresult.csv'

destination = 'C:\\Users\stephaniekendall\Flatiron\WaterQualityProject2'

dest = shutil.move(source,destination)

# os.rename("/Users/stephaniekendall/Downloads/biologicalresult.csv", 
#           "/Users/stephaniekendall/Flatiron/WaterQualityProject2/biologicalresult.csv")
# shutil.move("/Users/stephaniekendall/Downloads/biologicalresult.csv", 
#             "/Users/stephaniekendall/Flatiron/WaterQualityProject2/biologicalresult.csv")
# os.replace("path/to/current/file.foo", 
#            "/Users/stephaniekendall/Flatiron/WaterQualityProject2/biologicalresult.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\stephaniekendall\\Downloads\x08iologicalresult.csv'

In [ ]:
/Users/stephaniekendall/Downloads/biologicalresult.csv

/Users/stephaniekendall/Flatiron/WaterQualityProject2/biologicalresult.csv